In [27]:
%load_ext autoreload
%autoreload 2
from config.config import *
import numpy as np
import roboticstoolbox as rtb
import spatialmath as sm
import urchin
from urchin import URDF

import os


import swift


print(urchin.__version__)

urdf = URDF.load(os.path.join(os.getcwd(), PANDA_URDF))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.0.27
/home/vahid/sbu/Screw-Panda/models/franka_description/robots/franka_description/meshes/visual/link0.dae
/home/vahid/sbu/Screw-Panda/models/franka_description/franka_description/meshes/visual/link0.dae
/home/vahid/sbu/Screw-Panda/models/franka_description/meshes/visual/link0.dae
/home/vahid/sbu/Screw-Panda/models/franka_description/robots/franka_description/meshes/collision/link0.stl
/home/vahid/sbu/Screw-Panda/models/franka_description/franka_description/meshes/collision/link0.stl
/home/vahid/sbu/Screw-Panda/models/franka_description/meshes/collision/link0.stl
/home/vahid/sbu/Screw-Panda/models/franka_description/robots/franka_description/meshes/visual/link1.dae
/home/vahid/sbu/Screw-Panda/models/franka_description/franka_description/meshes/visual/link1.dae
/home/vahid/sbu/Screw-Panda/models/franka_description/meshes/visual/link1.dae
/home/vahid/sbu/Screw-Panda/models/franka_description/robot